In [10]:
import dspy
import os
from typing import Literal
import numpy as np
import polars as pl
import pandas as pd

import loader

In [2]:
api_key=os.getenv("OPENAI_API_KEY")

In [3]:
# lm = dspy.LM('openai/gpt-4o-mini', api_key=api_key)
lm = dspy.LM('ollama_chat/llama3.2', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

In [4]:
class Tagged(dspy.Signature):
    body_text: str = dspy.InputField()
    # input_tags: str = dspy.InputField()
    tag: Literal[
        "Side Effects"
        "Product Issues",
        "Account & Orders",
        "Customer Service",
        "Product Information",
        "System & Administrative"
    ] = dspy.OutputField()
    confidence: float = dspy.OutputField()

predict_tag = dspy.Predict(Tagged)

In [11]:
tickets = loader.get_tickets()
tickets

ticket_id,subject,body_text,tags
i64,str,str,list[str]
348342800,"""Shipment""","""Hello, I’m currently out of t…","[""general-product/safety"", ""complaint/heat""]"
348344318,"""issue with order -ORDER NO. #6…","""hi there! i placed an order an…","[""urgent"", ""internal/log"", … ""wcon-p""]"
343560310,"""Re: Order #579121 confirmed""","""Hello, I still haven’t receiv…","[""complaint/smell-taste"", ""complaint-date/2024-08-17"", … ""order/promo""]"
348376971,"""Fwd: A shipment from order #61…","""Hello, Please cancel this shi…","[""urgent"", ""refund/denied"", … ""wtr1-p""]"
348388616,"""Issue with pack""","""Hi, I ordered the first trim…","[""internal/log"", ""complaint/smell-taste"", … ""other/fishy""]"
…,…,…,…
368741112,"""786242 damaged product""",""" Sent from my iPhone""","[""complaint/damaged"", ""email""]"
368742742,"""Cancel subscription""","""Hi, I’d like to cancel my sub…","[""siena/stop-response"", ""cancel/no-reason"", … ""complaint-date/2024-12-31""]"
368762239,"""Return item""","""Return item -----------------…","[""#675926"", ""complaint/side-effects"", … ""offline-capture""]"


In [6]:
df = tickets.to_pandas()[:10]

In [7]:
# results = pd.Series(np.empty(10, dtype=str))
df['predicted_tag'] = pd.Series(np.empty(10, dtype=str))
df['confidence'] = pd.Series(np.empty(10, dtype=np.float64))

for index, row in df.iterrows():
    print(f"index: {index}")
    body = row['body_text']
    print(f"ticket id {row['ticket_id']}")
    prediction = predict_tag(body_text=row['body_text'])
    df.loc[index, 'predicted_tag'] = prediction.tag
    df.loc[index, 'confidence'] = prediction.confidence

index: 0
ticket id 348342800
index: 1
ticket id 348344318
index: 2
ticket id 343560310
index: 3
ticket id 348376971
index: 4
ticket id 348388616
index: 5
ticket id 348395199
index: 6
ticket id 348405986
index: 7
ticket id 348414135
index: 8
ticket id 348414173
index: 9
ticket id 348089071


df

In [8]:
df

,ticket_id,subject,body_text,tags,predicted_tag,confidence
0,348342800,Shipment,"Hello,\n\nI’m currently out of the country wil...","[general-product/safety, complaint/heat]",Customer Service,0.90
1,348344318,issue with order -ORDER NO. #612681,hi there! i placed an order and only part of t...,"[urgent, internal/log, complaint/quality, repl...",Account & Orders,0.90
2,343560310,Re: Order #579121 confirmed,"Hello, \nI still haven’t received my refund. A...","[complaint/smell-taste, complaint-date/2024-08...",Side EffectsProduct Issues,0.95
3,348376971,Fwd: A shipment from order #617343 is on the way,"Hello, \nPlease cancel this shipment and send ...","[urgent, refund/denied, no-csat, refund/courte...",Product Information,0.50
4,348388616,Issue with pack,"Hi, \n\nI ordered the first trimester pack and...","[internal/log, complaint/smell-taste, wtr1-p, ...",Side EffectsProduct Issues,0.90
5,348395199,Product Question,Product Question\n----------------------------...,"[complaint/smell-taste, complaint/nausea, wtr1...",Side EffectsProduct Issues,0.90
6,348405986,Vitamin,"Hello,\n\nI have tried this vitamin 2 days sin...","[internal/log, complaint/nausea, complaint-dat...",Side EffectsProduct Issues,0.90
7,348414135,Packet to pill swap,"Hello, \n\nI ordered the first trimester packe...","[complaint-date/2024-09-01, internal/log, #602...",Side EffectsProduct Issues,0.90
8,348414173,Re: QUESTION: ORDER NO. #605726,Are returns not an option? I feel like I waste...,"[replacement/courtesy, general-product/safety,...",Product Information,0.80
9,348089071,Order 598810,Recipient inbox full Your message couldn't be ...,"[internal/log, cancel/do-not-like, complaint/n...",Account & Orders,1.00


In [9]:
df.loc[9, 'body_text']

"Recipient inbox full Your message couldn't be delivered to . Their inbox is full, or it's getting too much mail right now. saand1289@gmail.com LEARN MORE (https://support.google.com/mail/?p=OverQuotaPerm) \n The response was: \n552 5.2.2 The recipient's inbox is out of storage space and inactive. Please direct the recipient to https://support.google.com/mail/?p=OverQuotaPerm 38308e7fff4ca-2f614e95d2bsor12618611fa.8 - gsmtp \nHi Sarah, \n\nHow would you rate the help Kim gave you? \n\n  (https://perelelhealth.gorgias.com/satisfaction-survey?token=IjM0ODA4OTA3MS42MjAxODc1ODYuMjE2NTc2NTMi.Gbalrw.P-25OvsfDZuLmwM3LtTvgepC32k&score=1)   (https://perelelhealth.gorgias.com/satisfaction-survey?token=IjM0ODA4OTA3MS42MjAxODc1ODYuMjE2NTc2NTMi.Gbalrw.P-25OvsfDZuLmwM3LtTvgepC32k&score=2)   (https://perelelhealth.gorgias.com/satisfaction-survey?token=IjM0ODA4OTA3MS42MjAxODc1ODYuMjE2NTc2NTMi.Gbalrw.P-25OvsfDZuLmwM3LtTvgepC32k&score=3)   (https://perelelhealth.gorgias.com/satisfaction-survey?token=IjM